# Import

In [1]:
import pandas as pd
import torch

from glcic.networks.discriminators import *
from glcic.utils import *

In [2]:
# load the dataset
batch_size = 4
train_dataset_dir = '../data/train/'
train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_dir, transform=torchvision.transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

In [3]:
# initialize networks
discriminator = Discriminator()
cn = CompletionNetwork()
cn.load("../logs/models/cn_scrapped_weights")
optimizer = torch.optim.Adadelta(discriminator.parameters())

Load: load_state dict from ../logs/models/cn_scrapped_weights


# Loads

In [4]:
# hyperparameters
num_batch = 10
replacement_val = torch.tensor([0.5062325495504219, 0.4255871700324652, 0.38299278586700136])

In [5]:
# last training session
loss_list, batch, resume_path, replacement_val = load_checkpoint(
        "../logs/phase2_checkpoints/", discriminator, optimizer
    )

In [7]:
# train
from glcic.trainers.discriminator_trainer import train_discriminator
loss_list = train_discriminator(discriminator, cn,
    optimizer,train_dataloader,
    num_batch, replacement_val)


### BATCH 1/10 ###
loss: 0.0 for non-completed images, 2.388148186149852e-11 for completed images

### BATCH 2/10 ###
loss: 0.0 for non-completed images, 8.944811918606445e-15 for completed images

### BATCH 3/10 ###
loss: 0.0 for non-completed images, 1.2461713605471658e-10 for completed images

### BATCH 4/10 ###
loss: 0.0 for non-completed images, 1.5563951194508263e-07 for completed images

### BATCH 5/10 ###
loss: 0.0 for non-completed images, 1.4428162558033364e-06 for completed images

### BATCH 6/10 ###
loss: 0.0 for non-completed images, 8.079596713894688e-11 for completed images

### BATCH 7/10 ###
loss: 0.0 for non-completed images, 1.9746452194734088e-13 for completed images

### BATCH 8/10 ###
loss: 0.0 for non-completed images, 4.3681232242409607e-11 for completed images

### BATCH 9/10 ###
loss: 0.0 for non-completed images, 1.217329839361958e-12 for completed images

### BATCH 10/10 ###
loss: 0.0 for non-completed images, 4.421854438163564e-09 for completed images


In [20]:
# batch prediction
lm,em = generate_mask(4)
next_batch = next(iter(train_dataloader))[0]
preds = discriminator(next_batch,lm)